# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686571


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:15,  2.70s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:02,  2.32s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:43,  1.67s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:28,  1.19s/it]

...

Rendering models:  73%|███████▎  | 22/30 [00:08<00:01,  6.56it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  7.89it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  8.77it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  9.50it/s]

Anfrily                               0.001186
not-logged-in-20a275b53abec8db9f41    0.015608
Footrot                               0.000662
shirogane                             0.000419
not-logged-in-0171e7c203c321ec131c    0.002653
cschwefl                              0.000490
Dynamatt                              0.000326
not-logged-in-f5ac928c4dcd4e9cb619    0.003946
rnevils                               0.000255
not-logged-in-a76dd994176a1ff14298    0.004854
DarioB                                0.000213
crush202020                           0.000327
pangeli5                              0.000832
not-logged-in-214814e52004b42491a4    0.002590
not-logged-in-c85269d75db88d20f0b0    0.007781
Lavadude                              0.038688
not-logged-in-6ea7dd94a335e5238780    0.002201
ts135992                              0.000260
w7250369                              0.001706
Bambura                               0.001423
the0retical                           0.000252
ktayl47      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())